In [1]:
import numpy as np
import pandas as pd
import random
import re
from collections import defaultdict

# 1. Hashing

## 1.1 Implementation of the hashing function

In [61]:
class HashLogLog:
    def __init__(self, bits=32):
        random.seed(42)
        self.bits = bits
        self.m = 2 ** self.bits
        self.p = 4999999937 # Prime number > 2^32
        self.a = random.randint(1, self.m)
        self.b = random.randint(0, self.m)
        assert(self.m < self.p), "Too many bits."
    
    def get_hash(self, x):
        x = int(x, 16)
        bin_hash = bin(((self.a * x + self.b) % self.p) % self.m)[2:]
        return bin_hash.zfill(self.bits)

## 1.2 HyperLogLog structure implementation

In [84]:
class HyperLogLog:
    def __init__(self, log2m, regwidth=32):
        self.log2m = log2m
        self.regwidth = regwidth
        self.my_hash = HashLogLog(bits=self.regwidth)
        self.m = 2 ** self.log2m
        assert(self.m in [16, 32, 64] or m >= 128)
    
    def structure(self, file_path):
        HLL = [0] * self.m
        with open(file_path) as f:
            for line in f:
                hashed = self.my_hash.get_hash(line)
                root = int(hashed[:self.log2m], 2)
                try:
                    temp = hashed[self.log2m:].index('1') + 1
                except:
                    temp = len(hashed[self.log2m:])
                if temp > HLL[root]:
                    HLL[root] = temp
        return HLL
        
    def cardinality(self, hll):
        d = {16: 0.673, 32: 0.697, 64: 0.709}
        if self.m >= 128:
            alpha = 0.7213 / (1 + 1.079 / self.m)
        else:
            alpha = d[self.m]
        
        return int(self.m ** 2 * alpha * (1 / sum([2**(-x) for x in hll])))

## 1.3 Sequentially adding data to the HLL

In [91]:
HLL = HyperLogLog(log2m=6, regwidth=32)

hll = HLL.structure('hash.txt')

print(hll)

[23, 26, 22, 26, 23, 21, 22, 23, 21, 21, 26, 24, 20, 21, 22, 21, 19, 21, 22, 24, 20, 24, 22, 22, 21, 20, 20, 22, 24, 26, 20, 24, 20, 21, 21, 23, 22, 20, 21, 23, 20, 21, 22, 21, 20, 24, 23, 22, 23, 21, 20, 21, 20, 21, 20, 20, 19, 24, 23, 21, 22, 25, 23, 23]


## 1.4 Cardinality and error of the filter

### 1.4.1 Cardinality

In [86]:
estimate = HLL.cardinality(hll)

estimate

103115574

### 1.4.2 Error

In [92]:
!sort hash.txt | uniq | wc -l

125000000
